In [122]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import aurora_data_api

In [141]:
nameMap = {
    'MEGAN': "Megan Schacht-bonneau",
    "JORDYS": "Jordys Placido",
    "NIKKY": "Nikky Voueth"
}

def getDate(inp):
    dateVal = re.findall('1?2/.*/.*', inp)[0]
    formatted = dt.datetime.strptime(dateVal, '%m/%d/%y').strftime('%Y-%m-%d')
    return formatted

def getFailureType(inp):
    if inp[0] == 'W':
        return inp[:6]
    else:
        return 'none'
    
def getBatchNumber(inp):
    return inp.split(' ')[1]

def getTester(inp):
    return nameMap.get(inp, inp)

def execute(q):
    try:
        db = aurora_data_api.connect(aurora_cluster_arn = r_ARN, secret_arn=s_ARN, database=dbName)
        c = db.cursor()
        c.execute(q) # Execute the query
        db.commit()
        return c.fetchall() # Display the output of said query
    except Exception as e: # Error occurred during execute
        print('Error with execute:', e)
    
        return (float('inf'), e)
    
def insertEntry(sn, bn, date, tst, ft, fail, msg, dt):
    try:
        q = ''' Insert into QAResults values ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')''' % (sn, bn, date, tst, ft, fail, msg, dt)
        return execute(q)
    except Exception as e:
        pass
        print('Issue with edit entry:', e)
    
    
def readPath(path, deviceType):
    df = pd.read_excel(path)
    df = df.dropna(how = 'all')
    df = df.drop(columns = ['Unnamed: 5'])
    df = df[df['Unnamed: 2'] != 'RESULT']
    df.columns = ['BatchNumber', 'SerialNumber', 'FunctionalTest', 'LogMessage', 'Tester']
    df['Tester'] = df['Tester'].apply(getTester)
    df['BatchNumber'] = df['BatchNumber'].apply(getBatchNumber)
    df['Date'] = df['LogMessage'].apply(getDate)
    df['FailureType'] = df['LogMessage'].apply(getFailureType)
    df['DeviceType'] = deviceType
    return df

# res1 = readPath('../../QCResults/Catalina2020/Catalina QC Test 12.20 12.26.xlsx', 'Catalina')
# res2 = readPath('../../QCResults/Catalina2020/Catalina QC Test 12.27 01.02.xlsx', 'Catalina')
# display(res1.head())
# display(res2.head())

In [144]:
port = 3306
user = 'admin'
password = 'password'

host = 'dashboarddb.cluster-cyvmmsfjcmrl.us-east-1.rds.amazonaws.com'
dbName = 'db'
r_ARN = 'arn:aws:rds:us-east-1:319481862133:cluster:dashboarddb'
s_ARN = 'arn:aws:secretsmanager:us-east-1:319481862133:secret:dashboardDBKey-6XBatB'
def insertTable(res):
    for i in range(len(res)):
        row = res.iloc[i]
        bn = row['BatchNumber']
        sn = row['SerialNumber']
        tst = row['Tester']
        msg = row['LogMessage']
        ft = row['FunctionalTest']
        fail = row['FailureType']
        date = row['Date']
        dt = row['DeviceType']
        insertEntry(sn, bn, date, tst, ft, fail, msg, dt)
    print('success!')
    

In [145]:
home = '../../QCResults/'
mini2020Path = home + 'Mini2020'
mini2021Path = home + 'Mini2021'
catalina2020Path = home + 'Catalina2020'
catalina2021Path = home + 'Catalina'
for fileName in os.listdir(mini2020Path):
    print(fileName)

MINI QC Test 11.22 11.28.xlsx
.DS_Store
MINI QC TEST 10.12 10.16.xlsx
MINI QC TEST 10.25 10.31.xlsx
MINI QC TEST 8.10 8.21.xlsx
MINI QC TEST 10.5 10.9.xlsx
MINI QC TEST 9.28 10.2.xlsx
MINI QC TEST 8.24 8.28.xlsx
MINI QC Test 11.29 12.5.xlsx
MINI QC Test 11.15 11.21.xlsx
MINI QC TEST 10.18 10.24.xlsx
MINI QC TEST 11.01 11.07.xlsx
MINI QC TEST 11.08 11.14.xlsx


In [146]:
res3 = readPath(mini2021Path + '/MINI QC Test 02.14 02.20 2021.xlsx', 'Mini')
res3.head()
insertTable(res3)

success!


In [147]:
res4 = readPath(mini2021Path + '/MINI QC Test 02.21 02.27 2021.xlsx', 'Mini')
display(res4.head())
insertTable(res4)

,BatchNumber,SerialNumber,FunctionalTest,LogMessage,Tester,Date,FailureType,DeviceType
1,NA,G8V1900004829P2A,PASS,Completed Qc testing 2/22/21,Jose A,2021-02-22,none,Mini
2,NA,G8V1900004829QMF,PASS,Completed Qc testing 2/22/21,Jose A,2021-02-22,none,Mini
3,NA,G8V1900004829SHR,PASS,Completed Qc testing 2/22/21,Jose A,2021-02-22,none,Mini
4,NA,G8V1900004829SHE,PASS,Completed Qc testing 2/22/21,Jose A,2021-02-22,none,Mini
5,NA,G8V1900004829PFV,PASS,Completed Qc testing 2/22/21,Jose A,2021-02-22,none,Mini


Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8V1900010464' for key 'PRI